# Regression Task

In [ ]:
!pip install polars[pandas]

In [1]:
import numpy
import pandas

## Data Cleaning and Processing

### Load Dataset from TXT File by IO Process

In [2]:

df = pandas.read_csv('../data/uci_gbm_data.txt', sep='   ', engine='python')
df.head()

,Lever position (lp) [ ],Ship speed (v) [knots],Gas Turbine shaft torque (GTT) [kN m],Gas Turbine rate of revolutions (GTn) [rpm],Gas Generator rate of revolutions (GGn) [rpm],Starboard Propeller Torque (Ts) [kN],Port Propeller Torque (Tp) [kN],HP Turbine exit temperature (T48) [C],GT Compressor inlet air temperature (T1) [C],GT Compressor outlet air temperature (T2) [C],HP Turbine exit pressure (P48) [bar],GT Compressor inlet air pressure (P1) [bar],GT Compressor outlet air pressure (P2) [bar],Gas Turbine exhaust gas pressure (Pexh) [bar],Turbine Injecton Control (TIC) [%],Fuel flow (mf) [kg/s],GT Compressor decay state coefficient.,GT Turbine decay state coefficient.
0,1.138,3.0,289.964,1349.489,6677.380,7.584,7.584,464.006,288.0,550.563,1.096,0.998,5.947,1.019,7.137,0.082,0.95,0.975
1,2.088,6.0,6960.180,1376.166,6828.469,28.204,28.204,635.401,288.0,581.658,1.331,0.998,7.282,1.019,10.655,0.287,0.95,0.975
2,3.144,9.0,8379.229,1386.757,7111.811,60.358,60.358,606.002,288.0,587.587,1.389,0.998,7.574,1.020,13.086,0.259,0.95,0.975
3,4.161,12.0,14724.395,1547.465,7792.630,113.774,113.774,661.471,288.0,613.851,1.658,0.998,9.007,1.022,18.109,0.358,0.95,0.975
4,5.140,15.0,21636.432,1924.313,8494.777,175.306,175.306,731.494,288.0,645.642,2.078,0.998,11.197,1.026,26.373,0.522,0.95,0.975


### Convert Any Format of Data to Numpy

In [3]:
df = df.to_numpy()
values = df[:,-1]

### Obtain Dataset Information: features.shape

In [4]:
features = df[:,:-1]
features.shape, values.shape # prefer 1d value, not (11934, 1)

((11934, 17), (11934,))

## Machine Learning Process

### Load Perming and Config Hyperparameters

In [5]:
import perming
main = perming.Box(17, 1, (30,), criterion='MSELoss', batch_size=4, activation='relu', inplace_on=True, solver='adam', learning_rate_init=0.01)
# main = perming.Regressier(17, (30,), batch_size=4, activation='relu', solver='adam', learning_rate_init=0.01)
# main = perming.COMMON_MODELS['Regression'](17, (30,), batch_size=4, activation='relu', solver='adam', learning_rate_init=0.01)
main.print_config()

MLP(
  (mlp): Sequential(
    (Linear0): Linear(in_features=17, out_features=30, bias=True)
    (Activation0): ReLU(inplace=True)
    (Linear1): Linear(in_features=30, out_features=1, bias=True)
  )
)


OrderedDict([('torch -v', '1.7.1+cu101'),
             ('criterion', MSELoss()),
             ('batch_size', 4),
             ('solver',
              Adam (
              Parameter Group 0
                  amsgrad: False
                  betas: (0.9, 0.99)
                  eps: 1e-08
                  lr: 0.01
                  weight_decay: 0
              )),
             ('lr_scheduler', None),
             ('device', device(type='cuda'))])

### Dataloader from Numpy with Multi-threaded

In [6]:
main.data_loader(features, values, random_seed=0)

### Training Stage and Accelerated Validation

In [7]:
main.train_val(num_epochs=2, interval=100, early_stop=True)

Epoch [1/2], Step [100/2387], Training Loss: 23.0912, Validation Loss: 24.5740
Epoch [1/2], Step [200/2387], Training Loss: 291.9099, Validation Loss: 6.7348
Epoch [1/2], Step [300/2387], Training Loss: 5637.1328, Validation Loss: 1480.3076
Epoch [1/2], Step [400/2387], Training Loss: 1211.0406, Validation Loss: 210.9741
Epoch [1/2], Step [500/2387], Training Loss: 90.4388, Validation Loss: 23.6573
Epoch [1/2], Step [600/2387], Training Loss: 67.0454, Validation Loss: 24.6701
Epoch [1/2], Step [700/2387], Training Loss: 1253.5343, Validation Loss: 1144.0096
Epoch [1/2], Step [800/2387], Training Loss: 39.3887, Validation Loss: 257.6939
Epoch [1/2], Step [900/2387], Training Loss: 0.9986, Validation Loss: 1.1887
Epoch [1/2], Step [1000/2387], Training Loss: 30.2453, Validation Loss: 9.7175
Epoch [1/2], Step [1100/2387], Training Loss: 264.4302, Validation Loss: 19.0528
Epoch [1/2], Step [1200/2387], Training Loss: 5.2984, Validation Loss: 8.8709
Epoch [1/2], Step [1300/2387], Training L

### Test with Trained Parameters

In [12]:
main.test()
# refer to https://pytorch.org/torcheval/main/ for metrics functional tools, like r2_score
# features_, values_ = torch.as_tensor(features, dtype=torch.float).to(torch.device("cuda")), torch.as_tensor(values, dtype=torch.float).to(torch.device("cuda"))
# pred = main.model(features_)
# r2_score(pred, values_)

loss of Box on the 1196 test dataset: 0.14259785413742065.


OrderedDict([('problem', 'regression'),
             ('loss',
              {'train': 0.18060052394866943,
               'val': 0.025247152894735336,
               'test': 0.14259785413742065})])

### Save Model Parameters to Models Folder

In [13]:
main.save(False, '../models/ucigbm.ckpt')

### Load Model Parameters from Models Folder

In [18]:
main.load(False, '../models/ucigbm.ckpt')

### Test with Loaded Parameters or Tune Model

In [19]:
main.test()
# main = perming.Box(17, 1, (25,), criterion='MSELoss', batch_size=4, activation='relu', inplace_on=True, solver='adam', learning_rate_init=0.01)
# main = perming.Regressier(17, (25,), batch_size=4, activation='relu', solver='adam', learning_rate_init=0.01)
# main = perming.COMMON_MODELS['Regression'](17, (25,), batch_size=4, activation='relu', solver='adam', learning_rate_init=0.01)

loss of Box on the 1196 test dataset: 0.14268778264522552.


OrderedDict([('problem', 'regression'),
             ('loss',
              {'train': 0.10978104919195175,
               'val': 0.7494950294494629,
               'test': 0.14268778264522552})])